In [1]:
import keras
import numpy as np
import pandas as pd
import cv2

Using TensorFlow backend.
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D, concatenate, Input, Lambda
from keras import utils
from keras.callbacks import Callback, LambdaCallback, EarlyStopping, ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization

In [3]:
def logger(epoch, logs):
    if epoch %2== 0: 
        print(epoch, logs)
logging_callback = LambdaCallback(
    on_epoch_end=logger)

In [4]:
# data2 is my own training images
import csv
lines = []
with open('data2/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '.') for x in line]
        lines.append(lineout)

In [5]:
images = []
stearing = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread('data2/' + img_path)
    images.append(image)
    stearing.append(line[3]) 

In [6]:
X_train = np.array(images)
y_train = np.array(stearing)

In [11]:
X_train_norm = (X_train / 255.) - 0.5

In [28]:
model = Sequential()
Lambda(lambda x: (x / 255.0) - 0.5)
model.add(Flatten(input_shape=X_train[0].shape))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, shuffle=True, epochs=6,validation_split=0.2, callbacks=[ stopping_callback])

Train on 6428 samples, validate on 1608 samples
Epoch 1/6
6428/6428 [==============================] - 4s 652us/step - loss: 5085274.4461 - val_loss: 3896.2098

In [29]:
model.save('model1_simple.h5')

In [8]:
def model2(ksize=(3,3), dropout=0.25):
    inputs = Input(shape=X_train[0].shape)
    processed = Lambda(lambda x: (x /255. - 0.5))(inputs)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv1')(processed)
    block1 = BatchNormalization()(block1)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv2')(block1)
    block1 = BatchNormalization()(block1)
    block1 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set1_pool')(block1)
    
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv2')(block2)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv3')(block2)
    block2 = BatchNormalization()(block2)
    block2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name='set2_pool')(block2)
    output2 = Flatten()(block2)
        
    fcblock = Dense(1024, activation='relu', name='fc1')(output2)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    fcblock = Dense(512, activation='relu' , name='fc2')(fcblock)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [9]:
K.clear_session()
batch_size = 8
epochs = 5
model = model2(ksize=(5,5))

history = model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/vgg_carsim1.h5', save_best_only=True)]
                   )

Train on 6428 samples, validate on 1608 samples
Epoch 1/5
6428/6428 [==============================] - 436s 68ms/step - loss: 0.7635 - val_loss: 0.0442
Epoch 2/5
6428/6428 [==============================] - 427s 66ms/step - loss: 0.0863 - val_loss: 0.0232

In [11]:
keras.__version__

'2.1.1'

In [14]:
def leNet(ksize=(3,3),dropout=0.25):
    inputs = Input(shape=X_train[0].shape)
#     processed = Lambda(lambda x: (x /255. - 0.5))(inputs)
    
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv1')(inputs)
    block1 = BatchNormalization()(block1)
    block1 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set1_pool')(block1)
    
    block2 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block2 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set2_pool')(block2)
    
    output2 = Flatten()(block2)
    fcblock = Dense(128, activation='relu', name='fc1')(output2)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    fcblock = Dense(84, activation='relu' , name='fc2')(fcblock)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [15]:
K.clear_session()
batch_size = 32
epochs = 5
lenet_model2 = leNet(ksize=(5,5))

lehistory2 = lenet_model2.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim2.h5', save_best_only=True)]
                   )

Train on 1764 samples, validate on 441 samples
Epoch 1/5
1764/1764 [==============================] - 16s 9ms/step - loss: 1.3489 - val_loss: 54.5161
Epoch 2/5
1764/1764 [==============================] - 14s 8ms/step - loss: 0.8405 - val_loss: 29.0661

In [17]:
lehistory2 = lenet_model2.fit(X_train, y_train, batch_size=batch_size,
                    epochs=10,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim2.h5', save_best_only=True)]
                   )

Train on 1764 samples, validate on 441 samples
Epoch 1/10
1764/1764 [==============================] - 14s 8ms/step - loss: 0.3600 - val_loss: 0.1150
Epoch 2/10
1764/1764 [==============================] - 13s 8ms/step - loss: 0.3039 - val_loss: 0.1059

### Data Augmentation

In [22]:
lines = []
with open('data3/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '') for x in line]
        lines.append(lineout)

images = []
stearing = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread('data3/' + img_path)
    images.append(image)
    stearing.append(line[3]) 
augmented_img = []
augmented_measurements = []

for img, measurement in zip(images, stearing):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)

In [27]:
X_train = np.array(augmented_img)
y_train = np.array(augmented_measurements)

In [29]:
K.clear_session()
batch_size = 32
epochs = 15
lenet_model = leNet(ksize=(5,5))

lehistory = lenet_model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim_aug.h5', save_best_only=True)]
                   )

Train on 3928 samples, validate on 982 samples
Epoch 1/15
3928/3928 [==============================] - 29s 7ms/step - loss: 1.1093 - val_loss: 0.3885
Epoch 2/15
3928/3928 [==============================] - 29s 7ms/step - loss: 0.5076 - val_loss: 0.1541